# P-ai AI/ML Workshop: Session 4

Welcome to P-ai's third session of the AI/ML workshop series! Today we'll learn about
- Deep learning
    - Intuition behind neural nets
    - How to build and train a neural net with Tensorflow and Keras
    - Types of neural nets
- Solving a real-world classification problem with a neural net

<img src="https://images.squarespace-cdn.com/content/5d5aca05ce74150001a5af3e/1580018583262-NKE94RECI46GRULKS152/Screen+Shot+2019-12-05+at+11.18.53+AM.png?content-type=image%2Fpng" width="200px">

## 1. Feed-forward Neural Nets

### A simple example

Let's say we have a text, and we want to predict whether the language of the text is English, Spanish, or German. One simple way to attempt this would be to calculate the frequency of each letter in the alphabet, and use those frequencies to predict the language. For this example, we'll only use the 26 standard English letters, although in practice, that would be a very dumb decision since the presence of letters like `ñ` or `ü` would pretty much immediately give away the answer.

So, our features would look like:
- $x_1$ frequency of the letter `a`
- $x_2$ frequency of the letter `b`  
...
- $x_{26}$ frequency of the letter `z`

To build a little bit of intuition for neural nets, let's say we saw this string of text:

In [ ]:
text = 'Mi abuela tiene noventa y cuatro años. Cuando era joven trabajó como enfermera en un hospital. Ahora le gustan las manualidades y hace pulseras para toda la familia. Por las mañanas sale a dar un paseo con sus amigas y por las tardes ve la televisión.'
# Remove special characters for the sake of example
text = text.replace('ñ', 'n')
text = text.replace('ó', 'o')
text

In [ ]:
# Helpful for counting
from collections import Counter
# Case doesn't matter
text = text.lower()
# Punctuation doesn't matter
text = text.replace(' ','')
text = text.replace('.','')
text = text.replace(',','')
text_len = len(text)
counter = Counter(text)
# Sort entries
counts = dict(sorted(counter.items(), key=lambda item: -item[1]))
print('Letter frequencies\n------------------')
for letter in counts:
    print(f"{letter}: {counts[letter] / text_len}")

It's not exactly trivial to guess which language this is from the frequencies alone, but you might have a bit of intuition. For example, `t` is extremely common in English (the second most frequent letter, actually), so we might suspect this isn't English given it's nearly halway down the frequency list. In other words, the weight for $x_{20}$ (the frequency of the letter `t`) should be higher for English than for Spanish.

If we assume that the probability of the text being a certain language is linearly related to these frequencies, we could frame this problem like a logistic regression problem:

$$
w_{1e}x_1 + w_{2e}x_2 + {...} + w_{26e}x_{26} + b_e = logit_e
$$

Where $x_1$ is the frequency of letter `a` in our text, $w_{1e}$ is its respective weight (for English), and ditto for the rest of the input varluables. $b_e$ is the bias.

> What's up with the logit?

You might remember from Workshop 2 that the output of a linear combination in linear regression is called a **logit**; it's related to the probability of the input data belonging to a certain class, but it's quite a probability yet (which should be apparent because it can be any real value, whereas probabilities should be between `0` and `1`). In logistic regression, we used the `sigmoid` to get from a logit to a probability, but we'll need to do something different for multiclass classification– we'll get to that in a second!

Notice that we'll have something very similar for Spanish and German, but with different weights and biases.

$$
w_{1s}x_1 + w_{2s}x_2 + {...} + w_{26s}x_{26} + b_s = logit_s \\
w_{1g}x_1 + w_{2g}x_2 + {...} + w_{26g}x_{26} + b_g = logit_g
$$

Note how every feature ($x_1$ through $x_{26}$) gets 'sent' to all three outputs. Conversely, each output receives all the inputs. We can sketch our neural net like this:

<img src="images/nn_1.png" width="500px">

Again, this is just a visual representation of the three equations we wrote above, we haven't made any leaps of logic. An arrow coming out of a neuron means to multiply its value with a weight. All the arrows entering a neuron means to sum all those products together. With that in mind, you should be able to see that we're just representing equations with neurons and arrows.

Also, notice the dimensions. Our input layer has shape `(26,)` (vector of size 26) and the output layer has shape `(3,)` (vector of size 3).

In [ ]:
# Illustrate the point by creating the input vector from the example above
import numpy as np
x = np.zeros(shape=(26,), dtype=np.float32)                # Create empty vector
for i, letter in enumerate('abcdefghijklmnopqrstuvwxyz'):  # Iterate through 26 letters
    if letter in counts:                                   # If the ith letter appeared in the text
        x[i] = counts[letter]           # Set x_i to that letter's count
x = np.divide(x, sum(x))                # Turn counts into frequencies
print("X_vector:\n", x)
print("Shape:", x.shape)

Notice how we turned an input (text, in this case) into a vector. This vector would then go into the neural net during training (which is how the model gets $x_1$ through $x_{26}$).

> What about the $y$ data? Why is it a vector of size 3?

For binary classification, our $y$ data can simply be a `1` or a `0`; either an input belongs to the class, or it does not. For multiclass classification, we think of having a separate probability for the input belonging to each class. Since our task has three languages we're trying to classify between, our output vector would have size 3. For example, the text we saw above was Spanish, so the correct $y$ would be `[0, 1, 0]`, assuming each value is the probability of the text being English, Spanish, and German, respectively (we could have picked any order; we just need to stick with one).

> Okay, what about that whole logit business?

Remember that, with linear regression with one output variable, we used the sigmoid function to turn a logit into a probability. For **multiclass classification**, we need to use a *generalized* version of the sigmoid function, which is called **softmax**. Softmax takes in *multiple* logits and transforms them such that each one lies in the range `[0,1]` and sum to `1`. That is, it turns your outputs into probabilities of the input belonging to each class!'

When a function is applied to the inputs of a layer, it's called an **activation function**. So, we apply an activation function of `softmax` on our output layer to turn our logits into probabilities. There are other activation functions we might apply to layers in our neural net. For example, you can actually think of "doing nothing" as the linear activation function ($y=x$). A very common one is `ReLU`, which just turns any negative number into `0`. There are plenty of others, such as `sigmoid` (which we've seen), `tanh`, and `leaky ReLU`. [Here's](https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/) a helpful guide to activation functions and what they're used for.

>So to review what we have in this architecture:

This is a **neural network** with an **input layer**, an **output layer**, and **no hidden layers**. The input layer has **26 neurons** and the output layer has **3 neurons**. There are **weights** (multiply with the input) that connect each node in first layer to the second layer, and **biases** (add with the input) for each neuron in the output layer. We apply the **softmax activation function** to the last layer of the neural net to output a probability vector.

<img src="images/nn_2.png" width="500px">

>So... how does it learn?

The model makes a certain number of predictions and calculates a **loss** (see Workshop 3 for a more detailed explanation of this). The model then uses **gradient descent** and a process called **backpropagation** (shortened to backprop or just BP) to iteratively adjust the weights and biases to improve the model's performance (minimize loss). Then, the model makes more predictions, and the whole process repeats.

In a little more detail:
1. Initially, the model's parameters are randomly or pseudo-randomly assigned
    - Two common initializers are [He](https://faroit.com/keras-docs/2.0.0/initializers/#he_uniform) and [Glorot](https://faroit.com/keras-docs/2.0.0/initializers/#glorot_uniform)
2. The model makes a prediction (or many); that just means running an input vector through the layers until a probability vector is output
    - See **batches**, explained below
3. The output vector (the prediction) is compared to the actual $y$ vector (e.g. `[0.2, 0, 0.8] <=> [0, 0, 1]`)
4. The model uses the loss function to calculate a **loss** (think error) from the predicted and actual output vectors
5. Using the loss, the model uses **backpropagation** to *propagate the error backwards through the layers*, adjusting the weights and biases along the way to hopefully decrease the loss
6. Steps 2-6 repeat until some stop condition is reached
    - For example, you may tell the model to train on all of the input data 5 times (5 **epochs**)
    - Another method is to automatically stop training when the validation loss stops decreasing

Backpropagation is a pretty cool process; it's not magic, it's vector calculus! I won't go into the details right now, but [here](https://medium.com/coinmonks/backpropagation-concept-explained-in-5-levels-of-difficulty-8b220a939db5) is a cool article that explains backprop in 5 degrees of in-depth-ness.

### Hidden layers

This is all well and good, but we may suspect that the probem is more complex than pure linear combinations. This is what leads us to add **hidden layers** between our input and output layers, which allow for the model to learn more complex relationships between variables. That would look something like this:

<img src="images/nn_3.png" width="800px">

>Again, note the architecture:

Everything is the same as the previous example, except we now have **two hidden layers**. In our code, we would say how many neurons each of these hidden layers has. For example, the first could have 128 and the second could have 256 (powers of 2 are common). All of these layers are **densely connected**, meaning every neuron is connected to every neuron in the next layer.

### Terminology summary

- `Neuron`: A neuron really just stores a value either from the input vector or preceding neuron(s), and then passes that value to the next neuron(s)
- `Layer`: A layer is a collection of neurons at the same "depth" of the neural network. This is usually easier seen than explained
    - `Input layer`: The first layer in a neural net; it takes in an input vector
    - `Hidden layer`: A layer between the input and output layer
    - `Output layer`: The last layer in a neural net; it produces the output of the net
- `Dense layer`: A layer in which every neuron is connected to every neuron in the previous layer
- `Weight`: A weight assigned to a pair of neurons in different layers. For example, if there are two densely connected layers, every neuron in the first layer is connected to a neuron in the second layer with a weight. The value of the first neuron gets multiplied by the weight and send to the second neuron.
- `Bias`: A value that gets added to every neuron in a layer; these values do not depend on the input
- `Trainable parameters`: The parameters of the model that are tuned during training; often, this means the weights and biases. Sometimes, people say "parameters" to mean the trainable parameters specifically
- `Hyperparameters`: Parameters of the model that aren't tuned through training, like the number of layers, number of neurons in each layer, learning rate, etc.
- `Loss`: A measure of "badness" comparing the model predictions to the actual outputs
- `Loss function`: A function that compares the predicted and actual outputs and produces a loss. There are a few loss functions that are very commonly used for certain problems. These are:
    - [Binary crossentropy](https://keras.io/api/losses/probabilistic_losses/#binary_crossentropy-function) for binary classification
    - [Categorical crossentropy](https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function) for multiclass classification
    - [Mean Squared Error](https://keras.io/api/losses/regression_losses/#mean_squared_error-function) for regression
    - Check out [https://keras.io/api/losses/](https://keras.io/api/losses/) for a complete list of Keras's loss functions!
- `Gradient descent`: The process by which neural nets (and other models) make predictions to estimate the loss gradient and then iteratively tweak parameters to decrease the loss
- `Learning rate`: This is like the step size of the gradient descent. A lower learning rate is generally more stable but requires more data to learn more, whereas a higher learning rate can potentially get to the local minimum faster, but might also "overshoot" it many times in the process.
- `Optimizer`: The method of gradient descent. Some common ones include [Adam](https://keras.io/api/optimizers/adam/) and [SGD](https://keras.io/api/optimizers/sgd/).
- `Backpropagation`: The process that describes *how* to update the weights and biases across the (potentially many) layers of a neural net. It's called *back* propagation because the error is propagated *backwards* through the layers
- `Epoch`: One complete "run through" of the training data. That is, if the model has been trained for 10 epochs, then it has seen every example in the training data 10 times
- `Batch`: A collection of examples that the model makes predictions on before backpropagating
    - `Batch size`: The size of each batch
    - `Stochastic gradient descent`: Essentially a batch size of 1; backprop after each prediction (most accurate)
    - `Minibatch gradient descent`: Split the training data into batches, backprop after each batch
    - `Batch gradient descent`: Treat the entire dataset as a batch; backprop after each epoch (most efficient)
    
You can read more about stochastic / minibatch / batch GD [here](https://sweta-nit.medium.com/batch-mini-batch-and-stochastic-gradient-descent-e9bc4cacd461)!

## Implementation: Intro to Tensorflow and Keras

<img src="https://3.bp.blogspot.com/-QZVBl08fmPk/XhO909Ha1dI/AAAAAAAACZI/q1a1UykGKe0KDUZ_ZITtWmM7bBJFRrvPQCLcBGAsYHQ/s1600/tensorflowkeras.jpg" width="500px">

You might be wondering how to actually build and train a neural net. The most popular framework for deep learning is Google's [Tensorflow](https://www.tensorflow.org/). Under the hood, it's basically a bunch of optimized graph algorithms that are necessary for neural networks.

While you can build a neural net with Tensorflow alone (and in the future, you might need to do this to create a more "customized" neural net), this can often be a bit more involved than it has to be for a beginner. Luckily, Google also developed [Keras](https://keras.io/), which is an API for Tensorflow; in other words, it lets you write more readable and intuitive code, and Keras takes care of the nitty-gritty Tensorflow-y details.

Let's take a look at how we would build the hypothetical neural net above with Keras.

In [ ]:
''' Imports; this might take a second to load '''
import tensorflow
# Import layers we need (in this case, just Dense)
from tensorflow.keras.layers import Dense
# Sequential model means you just add layers in a sequence
from tensorflow.keras.models import Sequential
# Adam optimizer
from tensorflow.keras.optimizers import Adam

In [ ]:
''' Build model '''
model = Sequential()                                   # Define sequential model
model.add(Dense(128, input_dim=26))                    # Input layer and first hidden layer
model.add(Dense(256))                                  # Second hidden layer
model.add(Dense(3, activation='softmax'))              # Output layer; note activation function

In [ ]:
''' Choose optimizer and loss function '''
opt = Adam(learning_rate=0.001)              # Set learning rate to 0.001
loss = 'categorical_crossentropy'            # Using categorical crossentropy for multiclass classification

In [ ]:
''' Compile model and print layer summary '''
model.compile(optimizer=opt, 
    loss=loss,
    metrics=['accuracy'])

model.summary()

And that's that! Now, let's imagine we have some data:

In [ ]:
import random
import numpy as np

# Generate enough random numbers
random_numbers = np.random.rand(1000 * 26)
# This represents 1,000 examples, each with 26 features (frequencies a-z)
X = np.reshape(random_numbers, (1000, 26))
# This represents 1,000 one-hot vectors (which language the corresponding x actually is)
y = np.zeros((1000, 3), dtype=int)
for i in range(len(y)):
    y[i][random.randint(0, 2)] = 1

In [ ]:
print("Fake dataset:")
print("X shape: ", X.shape)
print("First X:", X[0])
print("y shape: ", y.shape)
print("First y:", y[0])

Now, we can fit our model to this fake data.

In [ ]:
from sklearn.model_selection import train_test_split
# Split data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Here we can specify the number of epochs to train on, the batch size, and much more
# Check out the documentation for more!
history = model.fit(X_train, y_train, epochs=10, batch_size=32)

Cool! We see a lot of stuff being output, and it might seem like a lot; here's the breakdown:

<img src="images/tf_output.png" width="700px">

Notice how, as the model trained on the training data for more epochs, the loss decreased and the accuracy increased. This is to be expected; the more times the model sees the data, the more it can fit to that data. You should probably be concerned about something, though...

Since the data is random, the model should have not much more than a 33% accuracy. Yet, we see at the end of 10 epochs, the model's accuracy is above 40%. How can this be? Did the model discover some hidden patterns in the apparently random data?

No. This is the classic problem of **overfitting**. 

<img src="https://media.geeksforgeeks.org/wp-content/cdn-uploads/20190523171258/overfitting_2.png" width="600px">

Virtually every machine learning model has the tendency to overfit on its training data; that is, it begins to learn the *specifics* of the data instead of the *general trend*. This isn't good, because that means your model is **unstable**, which means (among other things) it *won't generalize well*. This is why we have training and test sets; we test our model on data it's never seen before to test whether it can actually generalize what it's learned, or if it overfit.

In [ ]:
# Test our model on the test dataset
model.evaluate(X_test, y_test)

The result of `evaluate` is a list of the final loss and accuracy (or whatever metric we compiled the model with). We can see that the model is actually only right ~33% of the time on data it hasn't seen before, which is exactly what we would expect. That means that our model did overfit on the training data a bit, as we would expect would happen with random data.

**Helpful tip**: When calling `.fit()`, you can also pass in a validation split, which splits your training data *again* into training and validation data, and Tensorflow will test your model on the validation data after each epoch.

In [ ]:
def buildModel():
    model = Sequential()
    model.add(Dense(128, input_dim=26))
    model.add(Dense(256))
    model.add(Dense(3, activation='softmax'))
    opt = Adam(learning_rate=0.001)
    loss = 'categorical_crossentropy'
    model.compile(optimizer=opt, 
        loss=loss,
        metrics=['accuracy'])
    return model

In [ ]:
model = buildModel()
# Use 20% of the data for validation
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

We can now see the validation loss and accuracy after each epoch and, as we would expect, the validation loss / accuracy don't get any better the more we train.

Before moving onto other types of neural nets and an example, we can quickly see how easy it is to make predictions with our neural net.

In [ ]:
# Dummy data; shape: (1, 26)
# That is, one example with 26 features
dataToClassify = np.reshape(np.random.rand(26), (1,26))
pred = model.predict(dataToClassify)
# Print prediction of first (only) example
print(pred[0])

Remember that, for multiclass classification, the output of the model is a vector of probabilities for each class. To get a single class, we can easily find the index with the highest probability.

In [ ]:
print(np.argmax(pred[0]))

So, our model predicts that this hypothetical input (which we imagine is a vector of letter frequencies) belongs to language 0 (say, English).

Pat yourself on the back, that's the basics of building a neural net!

## 2. Other neural nets!

There are many "flavors" of neural networks. Here are some of the most common:

### Multilayer Perceptron (MLP) aka "Vanilla" Neural Net
The neural nets we just looked at are MLPs, or the "vanilla" feed-forward neural net. Many times, people will refer to these "basic" neural nets as just "neural nets" because they're not specifying a more specific type. MLPs find a relationship between 1D input and 1D output. So, if you can encode both your inputs and outputs as vectors, chances are, all you need is an MLP.

### Convolutional Neural Network (CNN)
A CNN is a type of neural net that works best with input where the **location** of the values is important. Take our  vanilla neural net example; it doesn't matter which order we store the letter frequencies in our input vector so long as we stick with an order. We can't say the same for images; it may matter a lot *where* in an image some feature is. CNNs work by first **convolving** over the image, which would probably be a bit tangential to explain now, but you can read more about it [here](https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/). In any case, convolution allows the CNN to learn features about the relative location of values in the matrix, which is perfect for image processing!

<img src="https://miro.medium.com/max/1400/1*XbuW8WuRrAY5pC4t-9DZAQ.jpeg" width="600px">

### Long Short Term Memory (LSTM)
The LSTM is a type of **RNN** (Recurrent Neural Network), which means it's meant to deal with **temporal** data. For example, if you were trying to predict something about a **sequence** of values, a regular neural network wouldn't do the trick. The LSTM introduces mechanisms for the net to "remember" old information and synthesize it with new information (hence the name). For a long time, LSTMs were the go-to for learning language, since you can represent a series of words numerically (see [word embeddings](https://machinelearningmastery.com/what-are-word-embeddings/)!).

<img src="https://miro.medium.com/max/1400/1*ahafyNt0Ph_J6Ed9_2hvdg.png" width="600px">

### Transformer

Transformers have really taken off in the past few years, especially with the enormous success of language models like BERT and GPT. Transformers build off of another type of model called the **encoder-decoder**, where the model learns both how to encode and decode data to get from input to output; the classic example is language translation. First, you encode a series of words in one language into a vector (the "meaning"), which is then decoded into a series of words in the second language. The transformer is basically an encoder-decoder on steroids; it has stacks of encoders and stacks of decoders, and it also implements this cool concept called [Attention](https://arxiv.org/abs/1706.03762), which helps the model learn *which* parts of the input correspond to which parts of the output. Transformers pretty much totally outperform LSTMs on translation tasks!

<img src="https://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" width="600px">

### GAN

GAN stands for Generative Adversarial Network, and they were invented by machine learning celebrity [Ian Goodfellow](https://en.wikipedia.org/wiki/Ian_Goodfellow). The main idea of GANs is to have two components of the neural net; the *generator* and the *discriminator*, which "compete" with each other. The generator tries to generate data that is similar to the real data, and the discriminator tries to figure out which data are real and which were generated by the generator. Both "adversaries" get better throughout the training, and at some point, the generator may become so good, the discriminator can't tell which data is real and which is fake. At this point, thanks to the GAN, you have an impeccable generator and a classifier! There are lots of uses for GANs, from [generating images of people that don't exist](https://thispersondoesnotexist.com/) to [turning sketches into photorealistic images](https://arxiv.org/pdf/1801.02753.pdf).

<img src="https://i1.wp.com/bdtechtalks.com/wp-content/uploads/2018/05/GANs.png?resize=696%2C304&ssl=1" width="600px">

These are just a few of the most common "types" of models you'll hear about out there. There's no limit to the kinds of model you can put together, though. For example, if you need to learn spatio-temporal data, you can combine a CNN with an LSTM to get a CNN-LSTM. When you work with tensorflow directly, you can build your very own custom neural nets!

## 3. Case Study

To put our theory into practice, let's take a look at the [heart failure prediction dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data). The goal of this task is to predict whether patients died from heart failure within a certain time frame after their last check-in. First, we should take a look at the data we have.

In [ ]:
# Helpful imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
# Read and display data from file
heart_data = pd.read_csv('data/heart_failure.csv')
heart_data

We see that we have 12 features and one prediction variable, `DEATH_EVENT`. Some of the variables are continuous (like `age`, `platelets`), and others are binary (e.g. `anaemia`, `diabetes`). 

In [ ]:
CONTINUOUS_COLS = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
BINARY_COLS = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']
TARGET_COL = 'DEATH_EVENT'

Let's plot all of our continuous variables against the target variable.

In [ ]:
for col in CONTINUOUS_COLS:
    # Distribution plot
    sns.displot(heart_data, x=col, hue=TARGET_COL)
    plt.show()

We don't immediately see that any of these features would be excellent predictors of heart failure. The `time` variable (the follow-up period) might offer the most clues; it seems that a very quick follow-up results in heart failure more often than a much later follow-up.

We can also take a look at our binary variables; we'll plot them as pie graphs.

In [ ]:
for col in BINARY_COLS:
    f, (ax1, ax2) = plt.subplots(1, 2)
    print(col)
    ax1.pie(heart_data[heart_data[TARGET_COL] == 0][col].value_counts(), labels=[0,1], autopct='%1.1f%%')
    ax1.set_title('Survived')
    ax2.pie(heart_data[heart_data[TARGET_COL] == 1][col].value_counts(), labels=[0,1], autopct='%1.1f%%')
    ax2.set_title('Died')
    plt.show()

Across the board, we see little differences in the target variable due to any of these binary variables alone. Thus, we might expect this classification task be a bit challenging.

In [ ]:
# X data is everything but the target column
X_data = heart_data[heart_data.columns[:-1]]
# y data is the target column
y_data = heart_data[TARGET_COL]

# Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2)
print(f"{len(X_train)} training examples and {len(X_test)} test examples")

It's also worth noting that the *scale* of the continuous variables vary drastically. Take a look at this:

In [ ]:
print("Max age:", max(X_data['age']))
print("Max platelets:", max(X_data['platelets']))

This is generally not ideal for neural nets. A big difference in scale between different features means that the "larger scale" features will naturally overpower the "smaller scale" ones, and the net will need to learn extremely small / large weights to keep the features comparable. This is unnecessary work for the neural net when we can just **normalize** our input first.

There's a few different ways to normalize data, but the goal is to scale each features so they're comparable. We can use the [standard scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to perform standard normalization, which means transforming the data so that the mean and standard deviation of each feature is 0 and 1, respectively.

In [ ]:
def normalify(data, columns, scaler=None):
    ''' Apply normalization to the specified columns in data. Fits a scaler if one is not given '''
    # Get columns to be normalized
    data_to_normalize = data[columns]
    # Get remaining data (not to be normalized)
    remaining_cols = [c for c in data.columns if c not in columns]
    remaining_data = data[remaining_cols]
    # If no scaler is given, fit one
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(data_to_normalize)
    # Apply standard scaler
    data_normalized = scaler.transform(data_to_normalize)
    # Recombine normalized and remaining data, and also return the scaler
    return np.hstack((data_normalized, remaining_data)), scaler

In [ ]:
# Apply our normalization process
X_train_normalized, scaler = normalify(X_train, CONTINUOUS_COLS)
X_test_normalized, _ = normalify(X_test, CONTINUOUS_COLS, scaler=scaler)

In [ ]:
# For consistency, we'll also turn our y data into numpy arrays
y_train, y_test = np.array(y_train), np.array(y_test)

Our X data is now a matrix of values that mostly sit between -1 and 1. Cool!

In [ ]:
print("X training data:\n", X_train_normalized)
print("Shape:", X_train_normalized.shape)

And, as expected, our y data are binary 0s (no heart failure) and 1s (heart failure).

In [ ]:
print("y training data:\n", y_train)
print("Shape:", y_train.shape)

Let's build a model! We'll incorporate something called `Dropout`, which randomly deactivates (sets to 0) a certain proportion of weights in a layer. This might seem destructive; why do we want to un-learn what we've learned? Dropout is a great way to combat overfitting! 

By deactivating weights, the model is forced to re-learn those connections from the existing weights and in doing so, learns the data more deeply instead of "memorizing" the input-output relationship in the training data. You can think of dropout like working out; your muscles get damaged to grow back stronger.

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
def defineBaselineModel():
    ''' Define and return a neural net '''
    model = Sequential()
    model.add(Dense(64, input_dim=12))         # Input layer has 12 features; second layer of 64 neurons
    model.add(Dropout(0.3))                    # 30% of weights are deactivated
    model.add(Dense(128))                      # Third layer has 128 neurons
    model.add(Dropout(0.3))
    model.add(Dense(256))                      # Fourth layer has 256 neurons
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))  # One neuron for output with sigmoid activation
    opt = Adam(learning_rate=0.01)             # Define optimizer
    loss = 'binary_crossentropy'               # Binary crossentropy for loss
    model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
    return model

In [ ]:
model = defineBaselineModel()
model.summary()

In [ ]:
# Train model on data!
model.fit(X_train_normalized, y_train, batch_size=8, epochs=5)

In [ ]:
# And evaluate!
acc = model.evaluate(X_test_normalized, y_test)
print("Accuracy:", acc[1])

The performance will vary every time you train the model, but when I ran it, the test accuracy was 78%. Let's check how well the model would do if it guessed `0` every time.

In [ ]:
1 - (sum(y_data) / len(y_data))

So, that's a little bit reassuring; the model performs about 10% better than it would by just guessing that there is never a heart failure. This isn't stellar, but if we preprocessed the data better (e.g. apply a log scale to `creatinine_phosphokinase`), or if we had more data, we could hope for a better performance.

Even though we split our data into training and test data, it's very possible that, by random chance, our test set was "easy" and resulted in a better test accuracy than we would get with a different train-test split. For this reason, we often use **K-fold cross validation** as a way of combating this. Basically, the data gets split up into `k` "folds", and then the model is trained and tested on different combinations of those folds (each fold gets to be the test set once). We can take the average of each test accuracy as a more honest evaluation.

In [ ]:
from sklearn.model_selection import KFold

def create_data_splits(data, k=5):
    ''' Returns a list of k tuples (X_train, X_test, y_train, y_test) '''
    splits = []
    kf = KFold(n_splits=k, shuffle=True)
    # Get raw X and y data
    X_data = data[data.columns[:-1]]
    y_data = data[data.columns[-1]]
    # kf.split returns train and test indexes
    for train_indexes, test_indexes in kf.split(X_data):
        # Get actual data by "filtering" by index using pandas' iloc
        X_train, X_test = X_data.iloc[train_indexes], X_data.iloc[test_indexes]
        y_train, y_test = y_data.iloc[train_indexes], y_data.iloc[test_indexes]
        # Apply normalization
        X_train_normalized, scaler = normalify(X_train, CONTINUOUS_COLS)
        X_test_normalized, _ = normalify(X_test, CONTINUOUS_COLS, scaler=scaler)
        y_train, y_test = np.array(y_train), np.array(y_test)
        # Add tuple of data to splits
        splits.append((X_train_normalized, X_test_normalized, y_train, y_test))
    return splits

In [ ]:
# Use 10-fold validation
k = 10
splits = create_data_splits(heart_data, k=k)
accuracies = []
for X_train, X_test, y_train, y_test in splits:
    # Important: define a new model every time you train on a new data split!
    model = defineBaselineModel()
    # verbose=0 doesn't show the progress bar
    model.fit(X_train, y_train, batch_size=8, epochs=10, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(accuracy)
# Print average accuracy
print(sum(accuracies) / k)

When I ran this code, it resulted in an average accuracy around 80%, which isn't half bad, and substantially better than random guess.

That's all we have time for today! Another great thing to check out would be a confusion matrix, which will let you visualize *how* the model misclassifies. Check out workshop 3 if you'd like a refresher on how to make / interpret those.

## Closing remarks

<img src="images/working_on_workshop.jpg" width="400px">
<br />
<div width="100%" style="text-align: center">
    Me working on this workshop at midnight; hope you've enjoyed learning about machine learning!
</div>